<a href="https://www.kaggle.com/code/tridibraj/tomato-leaf-diseases-detection-cnn-95?scriptVersionId=184181253" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# Image parameters

In [ ]:

img_height, img_width = 256, 256  # Maintain original resolution
batch_size = 32
num_classes = 10

# Paths to dataset

In [ ]:
train_dir = '/kaggle/input/tomatoleaf/tomato/train'
val_dir = '/kaggle/input/tomatoleaf/tomato/val'

# Load and preprocess

In [ ]:
# Efficient Data Loading
AUTOTUNE = tf.data.experimental.AUTOTUNE

def preprocess(image, label):
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

#training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode='categorical'  # Use categorical labels
).map(preprocess).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

#validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode='categorical'  # Use categorical labels
).map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

# Model Architecture

In [ ]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the Model

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True, verbose=1)
]


history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks
)


# Evaluate the Model & Save

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Assuming history is your training history object
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.savefig('/kaggle/working/training_validation_accuracy.png')  # Save the accuracy plot

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.savefig('/kaggle/working/training_validation_loss.png')  # Save the loss plot

plt.show()

# Confusion Matrix
val_labels = []
val_preds = []

for images, labels in val_dataset:
    val_labels.extend(np.argmax(labels.numpy(), axis=1))
    val_preds.extend(np.argmax(model.predict(images), axis=1))

conf_matrix = confusion_matrix(val_labels, val_preds)

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('/kaggle/working/confusion_matrix.png')  # Save the confusion matrix plot

plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# Evaluate on validation dataset
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation accuracy: {val_acc:.2f}")

# Get the true labels and predictions
val_labels = []
val_predictions = []

for images, labels in val_dataset:
    preds = model.predict(images)
    val_predictions.extend(np.argmax(preds, axis=1))
    val_labels.extend(np.argmax(labels.numpy(), axis=1))

# Compute precision, recall, and F1 score
val_precision = precision_score(val_labels, val_predictions, average='weighted')
val_recall = recall_score(val_labels, val_predictions, average='weighted')
val_f1 = f1_score(val_labels, val_predictions, average='weighted')

print(f"Validation Precision: {val_precision:.2f}")
print(f"Validation Recall: {val_recall:.2f}")
print(f"Validation F1 score: {val_f1:.2f}")


In [ ]:
import os
import random
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('/kaggle/working/light_tomato.h5')

# Path to the validation directory
val_dir = '/kaggle/input/tomatoleaf/tomato/val'

# Get list of disease folders
disease_folders = os.listdir(val_dir)

# Class names
class_names = {
    0: 'bacterial spot',
    1: 'early blight',
    2: 'late blight',
    3: 'leaf mold',
    4: 'Septoria',
    5: 'spider mites',
    6: 'target spot',
    7: 'yellow leaf',
    8: 'mosaic',
    9: 'healthy'
}

# Iterate 10 times
for _ in range(10):
    print(f"Iteration {_ + 1}:")
    # Choose a random disease folder
    random_folder_name = random.choice(disease_folders)
    # Get list of images in the random disease folder
    images = os.listdir(os.path.join(val_dir, random_folder_name))
    # Choose a random image from the folder
    random_image = random.choice(images)
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(
        os.path.join(val_dir, random_folder_name, random_image),
        target_size=(img_height, img_width)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    img_array = img_array / 255.0  # Normalize
    # Predict disease label using the model
    predictions = model.predict(img_array)
    predicted_label = predictions.argmax(axis=-1)[0]
    # Update class_names dictionary if predicted_label is not in the dictionary
    if predicted_label not in class_names:
        class_names[predicted_label] = f'Class {predicted_label}'
    # Get the class name from the folder name
    actual_label = random_folder_name.split('___')[-1]
    # Print the predicted label and actual label
    print(f"Predicted: {class_names[predicted_label]}, Actual: {actual_label}")
    print()
